# Set-up

In [97]:
import json
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from jinja2.exceptions import UndefinedError
from pathlib import Path
import shutil
import requests
from urllib.parse import urlencode
from datetime import datetime, date

In [98]:
TEMPLATE_DIR_PATH = Path("../templates")
HOSTING_PATH = Path('../')
CONTENT_PATH = Path('../content')

Function to create proper HTML file names by replacing spaces with underscores

In [99]:
def page_link(a):
    """Return the HTML file name after replacing blank spaces(" ") with underscores("-")"""
    return a.replace(" ", "_") if " " in a else a

Creating an instance of the Environment class that looks for templates. Page_link is set to the global variable so that it can be accessed by all templates

In [100]:
environment = Environment(
    loader=FileSystemLoader(TEMPLATE_DIR_PATH),
    extensions=["jinja2.ext.loopcontrols", "jinja2.ext.do"],
)
environment.globals["page_link"] = page_link

In [101]:
def create_page(template, html, **kwargs):
    """
    Create an HTML page using a Jinja2 template and save it to a specified path.

    Parameters:
    ----------
    template : str
        The filename of the Jinja2 template to be used.
    html : str
        The filename of the HTML file to be generated.
    **kwargs : dict
        Additional keyword arguments to be passed to the Jinja2 template for rendering.

    Returns:
    -------
    None

    """
    page_template = environment.get_template(template)
    template_level = html.count("/")
    page_html_path = HOSTING_PATH / html
    page_html_path.parent.mkdir(parents=True, exist_ok=True)
    page_content = page_template.render(TEMPLATE_LEVEL=template_level, **kwargs)
    with open(page_html_path, mode="w", encoding="utf-8") as page:
        page.write(page_content)

In [102]:
personal_data = {}
for info_json_path in CONTENT_PATH.iterdir():
    with open(info_json_path, "r") as info_json_file:
        personal_data[info_json_path.stem] = json.load(info_json_file)


In [103]:
create_page(
    "base.html.j2",
    "index.html",
    personal_data=personal_data
)

## Research papers

In [104]:
ads_token = "FlShxfq7bR2Pw0CIUFiWW7CEYMQr9b4RHugVXDBO"
BASE_URL = "https://api.adsabs.harvard.edu/v1/search"
start = 0
my_articles = []
orcid_id = "0000-0002-0479-7235"
numFound = None
while numFound == None or start < numFound:
    query = {
        "q": f'orcid:"{orcid_id}"',
        "fl": "title, author,volume, page, bibcode, bibstem, year",
        "sort": "year desc",
        "start": start
    }
    encoded_query = urlencode(query)

    results = requests.get(f"{BASE_URL}/query?{encoded_query}", 
                        headers={'Authorization': 'Bearer ' + ads_token})
    numFound = results.json()['response']['numFound']
    articles = results.json()['response']['docs']
    fetched_rows = len(articles)
    start += fetched_rows
    for entry in articles:
        my_articles.append(entry)

In [106]:
df = pd.DataFrame(my_articles)
df

,bibcode,author,bibstem,page,title,volume,year
0,2024ApJ...966..135K,"[Kwok, Lindsey A., Siebert, Matthew R., Johans...","[ApJ, ApJ...966]",[135],[Ground-based and JWST Observations of SN 2022...,966,2024
1,2024A&A...681A..86M,"[Maltsev, K., Schneider, F. R. N., Röpke, F. K...","[A&A, A&A...681]",[A86],[Scalable stellar evolution forecasting. Deep ...,681,2024
2,2024ApJ...960...88S,"[Siebert, Matthew R., Kwok, Lindsey A., Johans...","[ApJ, ApJ...960]",[88],[Ground-based and JWST Observations of SN 2022...,960,2024
3,2024jwst.prop.5232K,"[Kwok, Lindsey, Andrews, Jennifer, Badenes, Ca...","[jwst, jwst.prop]",[5232],[Getting Late Early: Mid-Infrared Spectroscopy...,NaN,2024
4,2022zndo....596036B,"[Bradley, Larry, Sipőcz, Brigitta, Robitaille,...",[zndo],NaN,[astropy/photutils: 1.13.0],NaN,2024
...,...,...,...,...,...,...,...
140,2011MNRAS.410.1725T,"[Tanaka, Masaomi, Mazzali, Paolo A., Stanishev...","[MNRAS, MNRAS.410]",[1725],[Abundance stratification in Type Ia supernova...,410,2011
141,2010pese.confE..48K,"[Kerzendorf, Wolfgang, Schmidt, Brian, Yong, D...","[pese, pese.conf]",[48],[Progenitor searches in Historical Type SN Ia ...,NaN,2010
142,2010gama.conf..105E,"[English, Jayanne, Fiege, Jason, Wiegert, Ther...","[gama, gama.conf]",[105],[The DiVA's Mask: Iconifying Galaxies and Reve...,NaN,2010
143,2009ApJ...701.1665K,"[Kerzendorf, Wolfgang E., Schmidt, Brian P., A...","[ApJ, ApJ...701]",[1665],[Subaru High-Resolution Spectroscopy of Star G...,701,2009
